In [1]:
import numpy as np
import pandas as pd
from preprocessing.wrangling import get_indi_df, get_labels, slide_and_flatten
from preprocessing.extract_features import get_all_ta_features
from evaluation.eval import sliding_window_cv_regression, batch_test_swcv_regression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from numpy.lib.stride_tricks import sliding_window_view
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline

In [2]:
df = get_indi_df("ASHOKLEY.NS", ohlcvfile="data_collection/ohlcv_data/ohlcv_auto.csv", start_date="2017-01-01")
df = get_all_ta_features(df)
drop_columns = ['Date']
df.drop(drop_columns, axis=1, inplace=True)
move_dir_target, cls_target = get_labels(df['Close'])
df = df.iloc[:-1]
cls_target = cls_target.iloc[:-1]

/home/eee/anaconda3/lib/python3.9/site-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/eee/anaconda3/lib/python3.9/site-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [3]:
df10 = slide_and_flatten(df, window_len=10)
df30 = slide_and_flatten(df, window_len=30)
df60 = slide_and_flatten(df, window_len=60)


In [4]:
pipe = make_pipeline(
    (SelectFromModel(Lasso(max_iter=4000))),
    (RobustScaler()),
    (RandomForestRegressor(n_estimators=100))
)

In [5]:
y = cls_target - df['Close']
y10 = cls_target[9:] - df['Close'].iloc[9:]
y30 = cls_target[29:] - df['Close'].iloc[29:]
y60 = cls_target[59:] - df['Close'].iloc[59:]

def add_closing_price(y, cls_price):
    return y + cls_price

In [6]:
batch_test_swcv_regression(
    list_X = [df],
    list_y = [y],
    list_pipe = [pipe],
    list_n_tr = [30],
    list_n_ts = [1],
    scorers = [mean_squared_error,mean_absolute_percentage_error, r2_score],
    savefile='rfg_tests.csv',
    comment_X=["df"],
    list_post_processors=[(add_closing_price, {'cls_price':df['Close'].iloc[:len(df)-(30+1)]})]
)

# cls_price should have length len(X)-(n_tr+n_ts)

/home/eee/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9523731672412197, tolerance: 0.0035965743988255667
  model = cd_fast.enet_coordinate_descent(
/home/eee/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.047054832589193, tolerance: 0.009029475802634059
  model = cd_fast.enet_coordinate_descent(
/home/eee/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.97895615718209, tolerance: 0.009098175318421942
  model = cd_fast.enet_coordinate_descent(
/home/eee/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_de

3613     81.250311
3614     81.560626
3615     84.853271
3616     83.609709
3617     85.905836
           ...    
4862    132.442653
4863    138.432271
4864    136.336961
4865    131.608447
4866    133.429252
Name: Close, Length: 1254, dtype: float64 3613     81.849998
3614     82.100006
3615     83.500000
3616     84.549995
3617     85.900002
           ...    
4862    139.200005
4863    145.750000
4864    132.600006
4865    130.200005
4866    132.400002
Name: Close, Length: 1254, dtype: float64
A test completed. (Comment : ['df', 30, 1])


In [7]:
def first(a, **kwargs):
    second(a, **kwargs)

def second(a, f, **kwargs):
    b = f(a, **kwargs)
    print(b)

def tf(a, other):
    print("dfdfdf")
    return a + other

first(np.array([2, 4, 3]), **{'f': tf, 'other':np.array([-1, 0, 3])})


dfdfdf
[1 4 6]
